In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=22d6b44fcd261f9604564fc38f4710083d8c5de64b275a5a0a6d7a5061a45b79
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
from pyspark.sql import SparkSession
import time

def process_batch():
    # Read the JSON file into a DataFrame
    df = spark.read.option("multiline", "true").json(json_file_path)

    # Create a new column maintenance_priority
    df = df.withColumn(
        "maintenance_priority",
        (df["num_docks_disabled"] + df["num_vehicles_disabled"]) /
        (df["num_docks_available"] + df["num_vehicles_available"])
    )

    # Order the DataFrame in descending order of maintenance_priority
    df = df.orderBy("maintenance_priority", ascending=False)

    df.createOrReplaceTempView("station")

    try:

        # Save the DataFrame as a JSON file
        output_path = "/gdrive/My Drive/output.json"
        df.write.json(output_path,mode="overwrite")

    finally:
        # Unpersist the DataFrame to release resources
        df.unpersist()

# Create a Spark session
spark = SparkSession.builder.appName("BatchApp").getOrCreate()

# Specify the path to the JSON file
json_file_path = "/gdrive/My Drive/station_data.json"

# Set the interval in seconds
interval_seconds = 15

try:
    # Run indefinitely
    while True:
        # Record the start time for each iteration
        iteration_start_time = time.time()

        # Process the batch
        process_batch()

        # Calculate and print the time taken for the iteration
        iteration_end_time = time.time()
        iteration_elapsed_time = iteration_end_time - iteration_start_time
        print(f"Time taken for iteration: {iteration_elapsed_time:.2f} seconds")

        # Wait for the specified interval
        time.sleep(interval_seconds)

except KeyboardInterrupt:
    # Handle keyboard interrupt (e.g., press Ctrl+C to stop the loop)
    print("Stopping the application")

finally:
    # Stop the Spark session
    spark.stop()


Time taken for iteration: 1.64 seconds
Time taken for iteration: 0.99 seconds
Stopping the application


In [ ]:
spark_master = spark.conf.get("spark.master")
print(spark_master)

local[*]
